# Simple Baseline (LightGBM Version) — Hull Tactical Market Prediction

This notebook provides a **safe, fast and competition-compliant
baseline** for the Hull Tactical Market Prediction challenge on Kaggle.

### Key Features

-   LightGBM regression model (fast and stable)
-   Chronological split (no data leakage)
-   Median imputation + missing indicators
-   Volatility-based signal scaling
-   Kaggle-compatible inference server wrapper

In [ ]:
## Imports
import os
from pathlib import Path
# import math
import numpy as np
import pandas as pd
from typing import Tuple, Dict

# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import polars as pl

import warnings
warnings.filterwarnings('ignore')

In [7]:
## Configuration and Data Loading (kaggle_evaluation only)
# import kaggle_evaluation.default_inference_server as kdeval
# DATA_DIR = Path('/kaggle/input/hull-tactical-market-prediction')

## Configuration and Data Loading (local version only)
DATA_DIR = Path("01_data")

# Read CSV files from data_path
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH  = DATA_DIR / 'test.csv'

VALIDATION_SIZE = 2700          # days, approx. 30% of data
RANDOM_SEED = 42
VOL_MULTIPLIER_LIMIT = 1.2
VOL_WINDOW = 20

def time_split_train_val(df: pd.DataFrame, val_size: int = 2700):
    df = df.sort_values('date_id').reset_index(drop=True)
    train_df = df.iloc[:-val_size].copy()
    val_df   = df.iloc[-val_size:].copy()
    return train_df, val_df

train_raw = pd.read_csv(TRAIN_PATH)
test_raw  = pd.read_csv(TEST_PATH)
train_raw.shape, test_raw.shape

((8990, 98), (10, 99))

In [ ]:
## Feature Preparation
excluded = {'date_id', 'forward_returns', 'risk_free_rate', 'market_forward_excess_returns'}
feature_cols = [c for c in train_raw.columns if c not in excluded]
feature_cols = [c for c in feature_cols if c in test_raw.columns]

"""
The third line performs a crucial validation step by ensuring feature consistency between training and test datasets. 
It filters the feature list to include only columns that exist in both the training data and the test data 
(test_raw.columns). This step is essential because machine learning models require identical feature structures 
during training and prediction phases. If a feature exists in training data but not in test data, 
the model would fail during inference. 
This defensive programming approach prevents runtime errors and ensures that the model can successfully 
make predictions on the test set.

This two-step filtering process - first removing inappropriate columns, 
then ensuring train-test consistency - represents a best practice in machine learning pipelines. 
It creates a robust feature set that avoids data leakage while maintaining compatibility across different data splits, 
which is particularly important in time-series financial prediction tasks where the test set represents 
future market conditions.
"""

In [ ]:
def prepare_df(df: pd.DataFrame, median_map: Dict[str, float], feature_cols: list) -> pd.DataFrame:
    df = df.copy()
    for c in feature_cols:
        if c not in df.columns:
            df[c] = 0.0
            df[f'{c}_was_na'] = 1
            continue
        if df[c].dtype.kind in 'fiu':
            med = median_map.get(c, 0.0)
            was_na = df[c].isna().astype(int)
            df[c] = df[c].fillna(med)
            df[f'{c}_was_na'] = was_na
        else:
            df[c] = pd.to_numeric(df[c], errors='coerce')
            med = median_map.get(c, 0.0)
            was_na = df[c].isna().astype(int)
            df[c] = df[c].fillna(med)
            df[f'{c}_was_na'] = was_na
    return df

"""
This function implements a robust data preprocessing pipeline that handles missing values and data type inconsistencies 
while preserving information about missingness patterns - a crucial technique in machine learning feature engineering.

The function begins by creating a copy of the input DataFrame to avoid modifying the original data, 
following defensive programming principles. It then iterates through each feature column to apply consistent 
preprocessing steps. The first conditional check handles the case where an expected feature column 
is completely missing from the DataFrame. Rather than failing, it gracefully creates the missing column filled 
with zeros and immediately creates a corresponding indicator variable set to 1, 
signaling that this entire feature was absent. 
This approach ensures model compatibility across datasets with different column structures.

For columns that exist in the DataFrame, the function employs different strategies based on data type. 
The condition df[c].dtype.kind in 'fiu' checks if the column is already numeric (float, integer, or unsigned integer) 
using pandas' dtype kind codes. For numeric columns, it retrieves the pre-computed median from the median_map dictionary, 
creates a binary indicator tracking which values were originally missing, fills the missing values with the median, 
and stores the missingness indicator as a new feature column with the suffix _was_na.

The else clause handles columns that aren't recognized as numeric types, which commonly occurs with object columns 
containing mixed data types or string representations of numbers. The function uses pd.to_numeric() with errors='coerce' 
to attempt conversion to numeric format, where invalid values become NaN rather than causing errors. 
After this conversion attempt, it applies the same median imputation and missingness indicator creation 
process used for originally numeric columns.

This dual approach - median imputation combined with missingness indicators - is particularly valuable 
because it prevents information loss. The median provides a robust central tendency measure that's 
less sensitive to outliers than the mean, while the binary indicators allow the model to learn patterns 
related to missingness itself. In financial data, missing values often carry meaningful information 
(such as certain metrics not being available during market stress), 
making these indicator features potentially predictive. 
The function's comprehensive error handling and type conversion ensure it can process datasets 
with inconsistent formatting while maintaining feature consistency across training and test sets.
"""

In [9]:
## Train / Validation Split and Median Imputation
train_df, val_df = time_split_train_val(train_raw, val_size=VALIDATION_SIZE)

median_map = {c: float(train_df[c].median(skipna=True)) if train_df[c].dtype.kind in 'fiu' else 0.0 
              for c in feature_cols}

train_p = prepare_df(train_df, median_map, feature_cols)
val_p   = prepare_df(val_df, median_map, feature_cols)
test_p  = prepare_df(test_raw, median_map, feature_cols)

final_features = [f for c in feature_cols for f in (c, f"{c}_was_na")]
print("Number of features:", len(final_features))

Number of features: 188


In [10]:
final_features

['D1',
 'D1_was_na',
 'D2',
 'D2_was_na',
 'D3',
 'D3_was_na',
 'D4',
 'D4_was_na',
 'D5',
 'D5_was_na',
 'D6',
 'D6_was_na',
 'D7',
 'D7_was_na',
 'D8',
 'D8_was_na',
 'D9',
 'D9_was_na',
 'E1',
 'E1_was_na',
 'E10',
 'E10_was_na',
 'E11',
 'E11_was_na',
 'E12',
 'E12_was_na',
 'E13',
 'E13_was_na',
 'E14',
 'E14_was_na',
 'E15',
 'E15_was_na',
 'E16',
 'E16_was_na',
 'E17',
 'E17_was_na',
 'E18',
 'E18_was_na',
 'E19',
 'E19_was_na',
 'E2',
 'E2_was_na',
 'E20',
 'E20_was_na',
 'E3',
 'E3_was_na',
 'E4',
 'E4_was_na',
 'E5',
 'E5_was_na',
 'E6',
 'E6_was_na',
 'E7',
 'E7_was_na',
 'E8',
 'E8_was_na',
 'E9',
 'E9_was_na',
 'I1',
 'I1_was_na',
 'I2',
 'I2_was_na',
 'I3',
 'I3_was_na',
 'I4',
 'I4_was_na',
 'I5',
 'I5_was_na',
 'I6',
 'I6_was_na',
 'I7',
 'I7_was_na',
 'I8',
 'I8_was_na',
 'I9',
 'I9_was_na',
 'M1',
 'M1_was_na',
 'M10',
 'M10_was_na',
 'M11',
 'M11_was_na',
 'M12',
 'M12_was_na',
 'M13',
 'M13_was_na',
 'M14',
 'M14_was_na',
 'M15',
 'M15_was_na',
 'M16',
 'M16_was_na'

In [12]:
## LightGBM Training
train_data = lgb.Dataset(train_p[final_features], label=train_p['forward_returns'])
val_data   = lgb.Dataset(val_p[final_features], label=val_p['forward_returns'])

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 63,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'seed': RANDOM_SEED,
    'n_jobs': -1,
    'verbose': -1
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=2000,
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(100)]
)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.010955
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.0102081


In [13]:
## Volatility Scaling Calibration
def strategy_stats(returns, exposures):
    strat = exposures * returns
    mean = np.nanmean(strat)
    std  = np.nanstd(strat)
    sharpe = (mean / (std + 1e-9)) * np.sqrt(252)
    vol = std * np.sqrt(252)
    return {'sharpe': sharpe, 'vol': vol}

val_pred = model.predict(val_p[final_features], num_iteration=model.best_iteration)
market_vol = np.nanstd(train_p['forward_returns']) * np.sqrt(252)

best_k, best_sharpe = 0.1, -1e9
for k in np.linspace(0.01, 5.0, 100):
    exposures = np.clip((k * val_pred), 0, 2)
    stats = strategy_stats(val_p['forward_returns'], exposures)
    if stats['vol'] <= VOL_MULTIPLIER_LIMIT * market_vol and stats['sharpe'] > best_sharpe:
        best_k = k
        best_sharpe = stats['sharpe']

print(f"Chosen scaling factor k={best_k:.3f} with Sharpe={best_sharpe:.2f}")

Chosen scaling factor k=5.000 with Sharpe=0.50


In [ ]:
"""
This code implements a sophisticated portfolio optimization technique that calibrates 
the scaling of model predictions to maximize risk-adjusted returns while controlling 
portfolio volatility - a critical step in translating machine learning predictions into practical 
trading signals.

The strategy_stats function serves as the core evaluation engine for portfolio performance. 

It calculates strategy returns by multiplying exposures (position sizes) with actual market returns, 
effectively simulating the profit and loss of the trading strategy. 
The function computes the mean and standard deviation of these strategy returns, 
then derives two key metrics: the Sharpe ratio and annualized volatility. 
The Sharpe ratio calculation includes a small epsilon value (1e-9) in the denominator 
to prevent division by zero, and multiplies by the square root of 252 (trading days per year) 
to annualize the ratio. Similarly, the volatility is annualized by multiplying 
the standard deviation by √252, converting daily volatility to annual terms for easier interpretation.

The calibration process begins by generating model predictions on the validation set 
using the best iteration from the trained LightGBM model. It then calculates the market's 
baseline volatility by taking the standard deviation of training returns and annualizing it. 
This baseline serves as a reference point for controlling the strategy's risk exposure. 
The algorithm initializes tracking variables for the best scaling factor (best_k) 
and corresponding Sharpe ratio, starting with conservative values.

The heart of the optimization lies in the systematic search over scaling factors using 
np.linspace(0.01, 5.0, 100), which creates 100 evenly spaced values between 0.01 and 5.0. 
For each scaling factor k, the code multiplies the model predictions and clips the resulting 
exposures between 0 and 2, ensuring long-only positions with maximum 200% allocationation. 
This clipping prevents excessive leverage while allowing for concentrated positions 
when the model has high confidence.

The selection criteria embodied in the conditional statement represents sophisticated 
risk management. The algorithm only considers scaling factors that satisfy two conditions: 
the strategy's volatility must not exceed VOL_MULTIPLIER_LIMIT times the market volatility 
(typically 1.2x, allowing 20% more risk than the market), and the Sharpe ratio must improve 
upon the current best. This dual constraint ensures that the strategy doesn't take excessive 
risk while maximizing risk-adjusted performance. The approach is particularly valuable 
in financial applications where controlling downside risk is as important as maximizing returns, 
as it prevents the model from being over-aggressive in its position sizing while still capturing 
the predictive signal effectively.
"""

In [ ]:
## Test Predictions + Smoothing
test_pred = model.predict(test_p[final_features], num_iteration=model.best_iteration)

alpha = 0.8
smoothed_allocation = []
prev = 0.0
for x in np.clip(best_k * test_pred, 0, 2):
    s = alpha * x + (1 - alpha) * prev
    smoothed_allocation.append(s)
    prev = s
smoothed_allocation = np.array(smoothed_allocation)

submission_df = pd.DataFrame({
    'date_id': test_p['date_id'],
    'weight': smoothed_allocation
})
submission_df.to_csv("submission_lgb_fixed.csv", index=False)
print("Saved submission_lgb_fixed.csv")

Saved submission_lgb_fixed.csv


In [ ]:
"""
This code implements the final prediction and submission generation phase, incorporating temporal smoothing to create more stable portfolio allocations 
- a crucial technique for reducing transaction costs and improving real-world trading performance.

The process begins by generating raw predictions on the test dataset using the trained LightGBM model. 

The model.predict() call uses the best_iteration parameter to ensure predictions are made with the optimal number of boosting rounds 
determined during training, preventing overfitting. These raw predictions represent the model's assessment of expected returns 
for each date in the test set.

The smoothing mechanism employs an exponential moving average (EMA) with an alpha parameter of 0.8. 
This technique addresses a common problem in quantitative trading: raw model predictions often exhibit high volatility that 
would result in excessive portfolio turnover if implemented directly. The EMA formula s = alpha * x + (1 - alpha) * prev 
creates a weighted average between the current scaled prediction and the previous smoothed allocation. With alpha = 0.8, 
the current prediction receives 80% weight while the previous allocation contributes 20%, 
striking a balance between responsiveness to new signals and stability.

Before applying the smoothing, each prediction is scaled by the optimal factor best_k (determined during the volatility calibration phase) 
and clipped between 0 and 2 using np.clip(). This clipping enforces the constraint of long-only positions with maximum 200% allocation, 
preventing the model from suggesting impossible or overly risky position sizes. The clipping occurs within the loop's iteration variable, 
ensuring that each scaled prediction is properly bounded before entering the smoothing calculation.

The smoothing loop maintains state through the prev variable, which starts at 0.0 and gets updated with each smoothed allocation. 
This creates a temporal dependency where each allocation decision considers not just the current model prediction, 
but also the recent history of allocations. This approach mimics how professional portfolio managers gradually adjust positions 
rather than making dramatic changes, reducing market impact and transaction costs.

Finally, the code packages the results into a competition-ready submission format. 
The smoothed allocations are converted to a NumPy array for consistency, 
then combined with the corresponding date identifiers from the test set into a pandas DataFrame. 
The submission is saved as "submission_lgb_fixed.csv" with index=False to exclude row numbers, 
creating a clean two-column format that matches competition requirements. 
This systematic approach to prediction post-processing demonstrates sophisticated understanding of the practical challenges 
in translating machine learning signals into implementable trading strategies.
"""

In [ ]:
"""
Kaggle Evaluation Metric:

strategy_returns = risk_free_rate * (1 - position) + position * forward_returns

if position = 0 → invest in risk-free asset,

if position = 1 → invest like the market,

if position = 2 → you are leveraged ×2 on the market.
"""

In [ ]:
## Kaggle Inference Server Wrapper

_model = model
_best_k = best_k
_history_returns = list(train_p['forward_returns'].iloc[-VOL_WINDOW:].tolist())

def predict(pl_df: pl.DataFrame) -> float:
    global _history_returns
    pdf = pl_df.to_pandas()
    pdf_p = prepare_df(pdf, median_map, feature_cols)
    for f in final_features:
        if f not in pdf_p.columns:
            pdf_p[f] = 0.0
    x = pdf_p[final_features].to_numpy()
    pred = _model.predict(x, num_iteration=_model.best_iteration)[0]
    vol_est = np.std(_history_returns) or 1e-3
    allocation = float(np.clip((_best_k * pred) / (vol_est + 1e-9), 0, 2))
    if 'lagged_forward_returns' in pl_df.columns:
        try:
            _history_returns.append(float(pl_df['lagged_forward_returns'][0]))
        except:
            _history_returns.append(0.0)
    _history_returns = _history_returns[-VOL_WINDOW:]
    return alloc

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway((str(DATA_DIR),))

### Notebook Summary

| Feature        | Description                          |
|----------------|--------------------------------------|
| Model          | LightGBM (fast, robust)              |
| Validation     | Time-based (last 2700 days)          |
| Imputation     | Median + missing flags               |
| Signal control | Volatility scaling (Sharpe-based)    |
| Inference      | Kaggle-compatible `predict` function |
| Runtime        | \< 5 minutes on Kaggle GPU notebook  |